In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('graph.csv')


In [4]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


def my_strategy(df,start_date,end_date):
    ##strategy計算
    # 計算布林帶的中軌、上軌和下軌
    period = 14  # 布林帶期間大小
    df["SMA"] = df["close"].rolling(period).mean()
    df["std"] = df["close"].rolling(period).std()
    df["upper_band"] = df["SMA"] + 2 * df["std"]
    df["lower_band"] = df["SMA"] - 2 * df["std"]
    df["BB%B"] = (df["close"] - df["lower_band"]) / (df["upper_band"] - df["lower_band"])
    overbought_threshold = 1
    oversold_threshold = 0
    df["position"] = None
    
    #注意格式! buy、sell
    buy = {}  
    sell = [] 
    
    # 遍歷每一天的數據
    for i in range(2, len(df)):
        if df.loc[df.index[i-2], "BB%B"] <= oversold_threshold and df.loc[df.index[i-1], "BB%B"] > oversold_threshold :
            if df.loc[df.index[i-1], "position"] is None: 
                df.loc[df.index[i]:,"position"] = "long"
                stop_ls  = df.loc[df.index[i-1],"SMA"] - 3 * df.loc[df.index[i-1],"std"]
                stop_pt  = df.loc[df.index[i-1],"SMA"] + 3 * df.loc[df.index[i-1],"std"]  
                buy[df.index[i]] = (stop_ls, stop_pt)
                             
        if df.loc[df.index[i-1], "position"] == "long":
            
            if df.loc[df.index[i-1], "close"] <= stop_ls or df.loc[df.index[i-1], "close"] >= stop_pt :  
                df.loc[df.index[i]:,"position"] = None
                sell.append(df.index[i]) 
    
            else :
                # 超買反轉時賣出
                if df.loc[df.index[i-2], "BB%B"] >= overbought_threshold and df.loc[df.index[i-1], "BB%B"] < overbought_threshold : 
                    df.loc[df.index[i]:,"position"] = None
                    sell.append(df.index[i])

  
    return df, buy, sell
start_date = '2023-06-01 00:00:00'
end_date = '2023-11-12 00:00:00'
df, buy, sell = my_strategy(df,start_date,end_date)

In [5]:
buy

{89: (3706.6252786292584, 4065.2605768817743),
 121: (3815.632388096683, 4355.496709389764),
 167: (3879.8641244978935, 4556.209220514129),
 172: (3638.952921290239, 4596.657401855028),
 229: (3831.368027271184, 4195.205425710855),
 237: (3683.855284218395, 4125.956928505597),
 244: (3551.6615707259825, 4060.22454888173),
 272: (3125.1813603291125, 3812.3663510142255),
 321: (2692.033675205108, 3846.4545537943695),
 330: (1926.0663933159597, 3738.5928315584006),
 472: (2291.0116332301113, 2860.6492317984503),
 539: (2461.462298449851, 2929.609208054814),
 554: (2292.5384659990236, 2694.190945572375),
 606: (2258.9400966778594, 2524.381706581772),
 702: (2687.5724620943274, 2913.4089841659306),
 707: (2521.423833460237, 2989.28784981208),
 736: (2580.248645671806, 2901.576454256111),
 791: (2650.409581697268, 2893.3649408437063),
 794: (2518.5579405463573, 2967.4193753886343),
 812: (2317.9715775566583, 2631.841805861747),
 862: (2420.2760404703017, 2638.1327799808673),
 885: (2356.1058

In [6]:
sell

[104,
 126,
 171,
 182,
 236,
 243,
 262,
 320,
 327,
 425,
 490,
 543,
 573,
 614,
 705,
 725,
 745,
 793,
 798,
 837,
 883,
 889,
 894,
 935,
 1003,
 1029,
 1050,
 1054,
 1105,
 1124,
 1255,
 1344,
 1375,
 1388,
 1455,
 1474,
 1521,
 1540,
 1555,
 1581,
 1609,
 1628,
 1648,
 1655,
 1663,
 1694,
 1712,
 1758,
 1790,
 1792,
 1806,
 1820,
 1899,
 1948,
 1973,
 2144,
 2181,
 2203,
 2285,
 2361,
 2370,
 2435,
 2451,
 2472,
 2516,
 2610,
 2623,
 2657,
 2701,
 2785,
 2892,
 2926,
 2953,
 2994,
 3010,
 3073,
 3109,
 3126,
 3138,
 3216,
 3230,
 3240,
 3279,
 3292,
 3294,
 3303,
 3336,
 3357,
 3462,
 3483,
 3507,
 3535,
 3646,
 3714,
 3773,
 3793,
 3795,
 3865,
 3959,
 3992,
 4100,
 4133,
 4177,
 4188,
 4203,
 4212,
 4277,
 4298,
 4367,
 4430,
 4464,
 4510,
 4516,
 4530,
 4566,
 4604,
 4628,
 4637,
 4656,
 4680,
 4685,
 4709,
 4741,
 4774,
 4818,
 4843,
 4886,
 4907,
 4933,
 4949,
 4969,
 5047,
 5090,
 5112,
 5149,
 5162,
 5226,
 5232,
 5252,
 5371,
 5376,
 5411,
 5414,
 5418,
 5456,
 5484,
 5

In [4]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import backtest as bt

def test():

    # 設定回測參數(可更改)
    start_date = '2023-06-01 00:00:00'
    end_date = '2023-11-14 03:00:00'
    target = 10000 #初始投入金額
    investment_method = "simple_interest"
    
    #回測
    important_sheets, fee_return = bt.range_backtest(start_date,end_date,target,investment_method)
    bt.show_analysis(important_sheets,target,fee_return,investment_method)

test()



------分析結果------
投資金額 : 10000
PNL : 934.8894118843273
IL : -653.0077822351032
手續費收入 : 375.70451109245323
手續費報酬率 : 3.76%
註記:以下為幣對分析，不含手續費
總收益:2.82%
年化標準差：14.88%
最大回檔：-8.94%
夏普比率：0.52
索提諾比率：8.29
純持有策略分析
總收益:13.81%
年化標準差：22.96%
最大回檔：-10.60%
夏普比率：2.27
索提諾比率：50.47
